# Rheological Flow around Confined Cylinder

Results published:
[Kikker, Kummer, Oberlack: A fully coupled high-order discontinuous Galerkin solver for viscoelastic fluid flow](https://onlinelibrary.wiley.com/doi/10.1002/fld.4950), 
see also at [tubiblio](http://tubiblio.ulb.tu-darmstadt.de/125045/).

### Preliminaries

This example can be found in the source code repository as as `ConfinedCylinder_ConvergenceStudy.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Initialization tasks

Loading the `Rheology` namespace:

In [ ]:
using BoSSS.Application.Rheology;

Initialization of the Workflow management; there `ConfinedCylinder_ConvergenceStudy` is the project name which is used for all comutations (aka. sessions):

In [ ]:
BoSSSshell.WorkflowMgm.Init("ConfinedCylinder_ConvergenceStudy");

Overview on the available *Execution Queues* (aka. *Batch Processors*, aka. *Batch System*); these e.g. Linux HPC clusters on which compute jobs can be executed.
These queues are specified globally (i.e. for all projects for the current user on current machine)
in the file `~/.BoSSS/etc/BatchProcessorConfig.json`.
If `~/.BoSSS/etc/BatchProcessorConfig.json` is missing or un-readable, a default list of execution queues is defined (containing only the `MiniBatchProcessor` for executing on the current machine.

In [ ]:
ExecutionQueues

For this example (which is part of the BoSSS validation tests), a *default queue* is selected to run all jobs in the convergence study.
The default queue is defined by the following settings:
- A global (for curent user on current machine, for all projects) default queue is set 
  in the `~/.BoSSS/etc/BatchProcessorConfig.json` file, in the `DefaultQueueIndex` entry. 
  If no such entry is present, the default queue is queue no. 0.
- The default queue for each project can be overridden in the file `~/.BoSSS/etc/DefaultQueuesProjectOverride.txt`.
  
- Note: by intention, there is no means of overriding the default queue from within the worksheet. 
  This is, because worksheets should ideally be independent from the current machine.
  It is, however, possible to specify a queue for each job indipendently dureing the job activation (see below).


In [ ]:
GetDefaultQueue()

## Parameter set for Convergence Study

In [ ]:
int[] pOrder = new int[] {1, 2, 3, 4};
int numberGrids = 4;

In [ ]:
double[] WeissenbergS = new double[] { 0.0, 0.2 };

## Init grids and save to database

In [ ]:
IGridInfo[] grids = new IGridInfo[numberGrids];
//GridCommons grid;

// Function to specify the boundary condition in dependence of the position X:
Func<double[], string> edgeTagFunc = delegate (double[] X) {
    double x = X[0];
    double y = X[1];
    //  if (!C.FixedStreamwisePeriodicBC)
    //  {
        if (Math.Abs(x - (-15)) < 1.0e-10)
            return "Velocity_inlet";
        if (Math.Abs(x - (+15)) < 1.0e-10)
            return "Pressure_Outlet";
    //  }
    //  if (Math.Abs(y - (-2)) < 1.0e-10)
    //      return "Freeslip";
                        
        if (Math.Abs(y - (0)) < 1.0e-10)
            return "Freeslip";
                        
        if (Math.Abs(y - (+2)) < 1.0e-10)
            return "Wall_top";
                        
    //  if (-1.0 < y && y < 1.0 && -1.0 < x  && x < 1.0)
    //    return "Wall_cylinder";
                        
        if (0.0 < y && y < 1.0 && -1.0 < x  && x < 1.0)
            return "Wall_cylinder";
                        
        throw new ArgumentOutOfRangeException("at x = " + x + "and y = " +y);
    };

for (int k = 0; k < numberGrids; k++) { // loop over mesh resolutions...
    // import grid from Gmsh file:
    grids[k] = BoSSSshell.WorkflowMgm.ImportGrid(@"mesh_karman_OriginalBox_MEDIUM_" + k + "_half.msh", 
                                                 EdgeTagFunc:edgeTagFunc);
}

In [ ]:
grids

## Setup of control objects for a solver runs

In [ ]:
using BoSSS.Solution.AdvancedSolvers;

In [ ]:
// Code for Initial conditions
class MyInitCode {

static string nl = System.Environment.NewLine;
public static string InitCode(double Reynolds, double Weissenberg, double beta) => 
"static class InitialValues { " + nl +
" " + nl +
"    static double u0                   = 1.5; // 0.375;// 0.66;// 3 / 2;   " + nl +
"    static double h                    = 4; " + nl +
$"    public static double beta          = {beta}; " + nl +
$"    static double Reynolds             = {Reynolds}; " + nl +
$"    static double Weissenberg          = {Weissenberg};  " + nl +
"    " + nl +
"    public static double VelocityXfunction(double[] X) { " + nl +
"        return u0 * (1  - (X[1] *  X[1])/h); " + nl +
"    } " + nl +
"    " + nl +
"    public static double VelocityYfunction(double[] X) { " + nl +
"        return 0.0; " + nl +
"    } " + nl +
"    " + nl +
"    public static double Pressurefunction(double[] X) { " + nl +
"        return u0 * 0.5 * Reynolds * (35 - X[0]); " + nl +
"    } " + nl +
"    " + nl +
"    public static double StressXXfunction_wi02(double[] X) { " + nl +
"        const double Weissenberg = 0.2; " + nl +
"        return 2 * Weissenberg * (1 - beta) * u0 * (-2 / h) * X[1] * u0 * (-2 / h) * X[1]; " + nl +
"    } " + nl +
"    " + nl +
"    public static double StressXXfunction_wi00(double[] X) { " + nl +
"        const double Weissenberg = 0.0; " + nl +
"        return 2 * Weissenberg * (1 - beta) * u0 * (-2 / h) * X[1] * u0 * (-2 / h) * X[1]; " + nl +
"    } " + nl +
"    " + nl +
"    public static double StressXYfunction(double[] X) { " + nl +
"        return (1 - beta) * u0 * (-2 / h) * X[1]; " + nl +
"    } " + nl +
"    " + nl +
"    public static double StressYYfunction(double[] X) { " + nl +
"        return 0.0; " + nl +
"    } " + nl +
"}  " + nl;

}

In [ ]:
//MyInitCode.InitCode(1,2)

In [ ]:
List<RheologyControl> Controls = new List<RheologyControl>();

In [ ]:
Controls.Clear();
double _beta = 0.59;
double _Reynolds = 1.0;
foreach(double Weissenberg in WeissenbergS) { // loop over different Weissenberg numbers...
foreach(int degree in pOrder) { //               loop over polynomial orders...
int elemInd = 0;
foreach(var grd in grids) { //                   loop over grid resolutions...

    //Database
    RheologyControl C = new RheologyControl();
    C.savetodb        = true;
    C.SetGrid(grd);
    C.TracingNamespaces = "BoSSS,ilPSP";

    //SolverChooser
    C.NonLinearSolver.MaxSolverIterations  = 100;
    C.NonLinearSolver.MinSolverIterations  = 3;
    C.NonLinearSolver.ConvergenceCriterion = 0.0; // as accurate as possible, until no further reduction can be achieved.
    C.NonLinearSolver.SolverCode           = NonLinearSolverCode.Newton;
    C.LinearSolver = LinearSolverCode.direct_pardiso.GetConfig();

    //Timestepping
    C.TimesteppingMode   = AppControl._TimesteppingMode.Steady;
    C.Timestepper_Scheme = RheologyControl.TimesteppingScheme.ImplicitEuler;
    C.ObjectiveParam     = 1.0;

    //Configuration Shock capturing and body forces
    //C.UsePerssonSensor       = false;
    //C.SensorLimit            = 1e-4;
    C.AdaptiveMeshRefinement = false;
    C.RefinementLevel        = 10;
    //C.UseArtificialDiffusion = false;
    C.Bodyforces             = true;

    //Physical parameters
    C.beta                 = _beta;
    C.Reynolds             = _Reynolds;
    C.Weissenberg          = Weissenberg; //aim Weissenberg number!
    C.RaiseWeissenberg     = Weissenberg > 0.0;

    //Penalties
    C.ViscousPenaltyScaling = 1;
    C.Penalty2              = 1;
    C.Penalty1[0]           = 0.0;
    C.Penalty1[1]           = 0.0;
    C.PresPenalty2          = 1;  
    C.PresPenalty1[0]       = 0.0;
    C.PresPenalty1[1]       = 0.0;
    C.alpha                 = 1;  
    C.StressPenalty         = 1.0;

    //Create Fields
    C.SetDGdegree(degree);

    //Set initial values
    C.InitialValues.Add("VelocityX", new Formula("InitialValues.VelocityXfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    C.InitialValues.Add("VelocityY", new Formula("InitialValues.VelocityYfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    if(Weissenberg == 0.0)
        C.InitialValues.Add("StressXX", new Formula("InitialValues.StressXXfunction_wi00", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    else if(Weissenberg == 0.2)
        C.InitialValues.Add("StressXX", new Formula("InitialValues.StressXXfunction_wi02", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    else
        throw new NotImplementedException("Missing Weissenberg = " + Weissenberg);    
    C.InitialValues.Add("StressXY", new Formula("InitialValues.StressXYfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    C.InitialValues.Add("StressYY", new Formula("InitialValues.StressYYfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta)));

    //Set Boundary Conditions
    //C.AddBoundaryValue("Wall_top", "VelocityX", Wall);
    //C.AddBoundaryValue("Wall_top", "VelocityY", Wall);
    //C.AddBoundaryValue("Wall_cylinder", "VelocityX", Wall);
    //C.AddBoundaryValue("Wall_cylinder", "VelocityY", Wall);

    C.AddBoundaryValue("Velocity_inlet", "VelocityX", new Formula("InitialValues.VelocityXfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    C.AddBoundaryValue("Velocity_inlet", "VelocityY", new Formula("InitialValues.VelocityYfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    if(Weissenberg == 0.0)
        C.AddBoundaryValue("Velocity_inlet", "StressXX", new Formula("InitialValues.StressXXfunction_wi00", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    else if(Weissenberg == 0.2)
        C.AddBoundaryValue("Velocity_inlet", "StressXX", new Formula("InitialValues.StressXXfunction_wi02", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    else
        throw new NotImplementedException("Missing Weissenberg = " + Weissenberg);
    C.AddBoundaryValue("Velocity_inlet", "StressXY", new Formula("InitialValues.StressXYfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    C.AddBoundaryValue("Velocity_inlet", "StressYY", new Formula("InitialValues.StressYYfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta))); 
            
    //Save Session and next...
    C.SessionName = $"ConfinedCylinder_ConvergenceStudy_p{degree}_meshNo{elemInd}_Wi{Weissenberg}_half";
    Controls.Add(C);
    Console.WriteLine("Created control: " + C.SessionName);
    elemInd++;
}
}
}

In [ ]:
wmg.Sessions

## Launch Jobs

In [ ]:
Controls.Count

In [ ]:
foreach(var ctrl in Controls) {
   var oneJob              = ctrl.CreateJob();
   oneJob.NumberOfMPIProcs = 1;
   oneJob.Activate();     
}

In [ ]:
// wait for all jobs to finish (up to 5 days, check every 5 minutes)
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(TimeOutSeconds:(3600*24*5), PollingIntervallSeconds:(60*5));

In [ ]:
// detect failed Jobs in the job management
var suspects = BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Value)
    .Where(job =>    job.LatestSession == null 
                  || job.LatestSession.Tags.Contains(SessionInfo.NOT_TERMINATED_TAG)
                  || job.LatestSession.Tags.Contains(SessionInfo.SOLVER_ERROR)).ToArray();
suspects

In [ ]:
NUnit.Framework.Assert.IsTrue(suspects.Count() <= 0, $"{suspects.Count()} Failed Jobs of {BoSSSshell.WorkflowMgm.AllJobs.Count()} in total.");

### Inspect the output of some arbitrary job:

In [ ]:
BoSSSshell.WorkflowMgm.AllJobs.First().Value.Stdout

In [ ]:
BoSSSshell.WorkflowMgm.AllJobs.First().Value.Stderr